<a href="https://colab.research.google.com/github/berkaykarlik/whoIsThatPokemon_CNN/blob/master/Atesv2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Mount the Drive

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


Import Libraries

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import random
from tqdm import tqdm



Get the Images

In [4]:
path='/content/gdrive/My Drive/PokemonData'

classes = []
data = []
SIZE = 100

def get_data():
  class_no = 0
  for dir in tqdm(os.listdir(path)):
    classes.append(dir)
    dir_path = os.path.join(path,dir)
    for img in tqdm(os.listdir(dir_path)):
      try:
        img_array = cv2.imread(os.path.join(dir_path,img))
        resized_array = cv2.resize(img_array,(SIZE,SIZE))
        data.append([resized_array,class_no])
      except:
        pass
    class_no += 1
get_data()

Streaming output truncated to the last 5000 lines.
100%|██████████| 150/150 [41:10<00:00, 16.47s/it]


Seperate features& labels, convert feature set to numpy array



In [0]:
x = []
y = []
for features,label in data:
    x.append(features)
    y.append(label)

x = np.array(x).reshape(-1, SIZE, SIZE, 3)
y = np.array(y)

Split Data

In [106]:
test_images = []
test_labels = []
train_images = []
train_labels = []


testCnt = 0
for i in range(1,len(y)):
    if y[i] == y[i-1]:
        if testCnt < 5:
            test_images.append(x[i-1])
            test_labels.append(y[i-1])
        else:
            train_images.append(x[i-1])
            train_labels.append(y[i-1])
        testCnt = testCnt + 1
    else:
        testCnt = 0
    
test_images = np.array(test_images).reshape(-1, SIZE, SIZE, 3)
train_images = np.array(train_images).reshape(-1, SIZE, SIZE, 3)
test_labels = np.array(test_labels)
train_labels = np.array(train_labels)


print(train_images.shape,test_images.shape)

(5935, 100, 100, 3) (750, 100, 100, 3)


CNN

In [0]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping


In [0]:
train_images, test_images = train_images / 255.0, test_images / 255.0

In [0]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)

In [0]:
model = Sequential([
    Conv2D(32, 3, padding='same', activation='relu', input_shape=(SIZE, SIZE ,3)),
    Conv2D(64, 3, padding='same', activation='relu',input_shape=(SIZE, SIZE ,3)),
    MaxPooling2D(pool_size=(5, 5),strides=(3, 3), padding='valid'),
    Conv2D(64, 3, padding='same', activation='relu'),
    MaxPooling2D(pool_size=(5, 5),strides=(3, 3), padding='valid'),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(150)
])

In [111]:
model.compile(optimizer='adagrad',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'] )
history = model.fit(train_images, train_labels, epochs=150, 
                    validation_data=(test_images, test_labels), callbacks=[callback])

Epoch 1/150
186/186 [==============================] - 3s 16ms/step - loss: 4.9006 - accuracy: 0.0239 - val_loss: 4.5794 - val_accuracy: 0.0680
Epoch 2/150
186/186 [==============================] - 3s 15ms/step - loss: 4.3219 - accuracy: 0.0785 - val_loss: 3.9864 - val_accuracy: 0.1387
Epoch 3/150
186/186 [==============================] - 3s 15ms/step - loss: 3.8819 - accuracy: 0.1321 - val_loss: 3.6038 - val_accuracy: 0.2320
Epoch 4/150
186/186 [==============================] - 3s 15ms/step - loss: 3.5851 - accuracy: 0.1757 - val_loss: 3.3509 - val_accuracy: 0.2653
Epoch 5/150
186/186 [==============================] - 3s 15ms/step - loss: 3.3474 - accuracy: 0.2199 - val_loss: 3.1343 - val_accuracy: 0.3160
Epoch 6/150
186/186 [==============================] - 3s 15ms/step - loss: 3.1486 - accuracy: 0.2586 - val_loss: 2.9759 - val_accuracy: 0.3413
Epoch 7/150
186/186 [==============================] - 3s 15ms/step - loss: 2.9701 - accuracy: 0.2992 - val_loss: 2.8432 - val_accuracy: